In [1]:
"""
This notebook is used for model hyper-parameter searching.
"""
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib
import matplotlib.pyplot as plt
from pprint import pprint
from typing import Dict, List

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
import sys
sys.path.append("../")
# If this notebook file is not placed under in /notebook/ directory,
# adding directory "../" might not correly add the project directory.
# If adding "../" does not solve the importing problem, we need to setup 
# the directory mannually.
try:
    import constants
except ModuleNotFoundError:
    core_dir = input("Directory of core files >>> ")
    if not core_dir.endswith("/"):
        core_dir += "/"
    sys.path.append(core_dir)
    import constants

In [3]:
from core.tools.metrics import *
import core.tools.visualize as visualize
from core.tools.time_series import *
from core.tools.data_import import *
import core.tools.rnn_prepare as rnn_prepare

import core.models.stacked_lstm as stacked_lstm

import core.training.hps_methods as hps_methods

In [4]:
# data preparation phase.
pprint(constants.DATA_DIR)
# choice = None
# while choice is None or choice not in constants.DATA_DIR.keys():
#     if choice is not None:
#         print("Invalid data location received, try again...")
#     choice = input("Select Dataset >>> ")
choice = "a"

FILE_DIR = constants.DATA_DIR[choice]

print(f"Dataset chosen: {FILE_DIR}")

print("Avaiable configuration files found: ")
for cf in os.listdir("../hps_configs"):
    if cf.endswith("config.py"):
        print("\t" + cf)

# config_name = input("Select config file >>> ")
config_name = "mac_config"
if config_name.endswith(".py"):
    config_name = config_name[:-3]

exec(f"import hps_configs.{config_name} as config")

print("Reading configuration file...")
for att in dir(config):
    if att.endswith("_config"):
        print(f"\tLoading: {att}")
        exec(f"globals().update(config.{att})")

{'a': '/Users/tianyudu/Documents/Academics/EconForecasting/AnnEconForecast/data/UNRATE.csv',
 'b': '/home/ec2-user/AnnEconForecast/data/UNRATE.csv',
 'c': '/home/ec2-user/AnnEconForecast/data/DEXCAUS.csv'}
Dataset chosen: /Users/tianyudu/Documents/Academics/EconForecasting/AnnEconForecast/data/UNRATE.csv
Avaiable configuration files found: 
	sample_config.py
	ec2_config.py
	mac_config.py
Reading configuration file...
	Loading: dp_config


In [ ]:
parameter_collection = hps_methods.gen_hparam_set(
    config.train_param
)

Total number of parameter sets generated: 3


In [ ]:
pprint(parameter_collection[1])

{'clip_grad': None,
 'epochs': 150,
 'fig_path': '/Users/tianyudu/Desktop/sample_model/model_figs/num_neurons=(32, '
             '64)-learning_rate=0.1',
 'hparam_str': 'num_neurons=(32, 64)-learning_rate=0.1',
 'learning_rate': 0.1,
 'model_path': '/Users/tianyudu/Desktop/sample_model/saved_models/num_neurons=(32, '
               '64)-learning_rate=0.1',
 'num_inputs': 1,
 'num_neurons': (32, 64),
 'num_outputs': 1,
 'num_time_steps': 12,
 'report_periods': 10,
 'tensorboard_dir': '/Users/tianyudu/Desktop/sample_model/tensorboard/num_neurons=(32, '
                    '64)-learning_rate=0.1'}


In [ ]:
class obj():
    def __init__(self, d):
        for k, v in d.items():
            setattr(self, k, v)

z = {"a": 1, "b": 2, "c": 3}
o = obj(z)

In [ ]:
# Check here.

In [ ]:
for (i, param) in enumerate(parameter_collection):
    print("================================================================")
    print(f"Executing hyper-parameter searching session [{i}/{len(parameter_collection) - 1}]...")
    print("Session Config:\n\t" + param["hparam_str"].replace("-", "\n\t"))
    start = datetime.now()
    hps_methods.individual_train(
        model_param=param,
        data_param=config.dp_config,
        exec_core=stacked_lstm.exec_core,
        file_dir=FILE_DIR
    )
    print(f"Time taken for session [{i}]: {str(datetime.now() - start)}.")

Executing hyper-parameter searching session [0/2]...
Session Config:
	num_neurons=(16, 32)
	learning_rate=0.1
Dataset loaded.    
	Index type: datetime64[ns]    
	Data type: float64
StandardScaler applied, scaling based on the first 679 observations.
Total 836 observations generated.
Note: shape format: (num_obs, time_steps, num_inputs/outputs)
X shape = (836, 12, 1), y shape = (836, 1, 1)
Training and testing set generated,        
X_train shape: (668, 12, 1)        
y_train shape: (668, 1)        
X_test shape: (84, 12, 1)        
y_test shape: (84, 1)        
X_validation shape: (84, 12, 1)        
y_validation shape: (84, 1)
Saving the model...
Time taken for [150] epochs:  0:00:04.164120
Final result:
Loss Summary:
	mae=0.5276699662208557
	mse=0.43354907631874084
	rmse=0.6584444642066956
	mape=6.428924560546875
Time taken for session [0]: 0:00:05.898569.
Executing hyper-parameter searching session [1/2]...
Session Config:
	num_neurons=(32, 64)
	learning_rate=0.1
Dataset loaded.   